In [1]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter nbextension install --py widgetsnbextension --sys-prefix

In [54]:
from gradio_client import Client #, handle_file
import os

In [55]:
image_directory = 'C:/Users/14122/Desktop/test data/Met-Meme/Eimages'
#image_directory = 'C:/Users/14122/Desktop/test data/Met-Meme/test_data_En_ver'


image_files = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [63]:
image_directory_1 = 'C:/Users/14122/Desktop/test data/Met-Meme/Eimages'

In [65]:
import os

def count_images(image_directory, extensions=['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']):
    count = 0
    for root, dirs, files in os.walk(image_directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                count += 1
    return count

# 统计图像文件数量
image_file_count = count_images(image_directory_1)

print(f"图像文件数量: {image_file_count}")

图像文件数量: 3995


In [56]:
image_files

['image_ (0).jpg',
 'image_ (1).jpg',
 'image_ (10).jpg',
 'image_ (100).jpg',
 'image_ (1000).jpg',
 'image_ (1001).jpg',
 'image_ (1002).jpg',
 'image_ (1003).jpg',
 'image_ (1004).jpg',
 'image_ (1005).jpg',
 'image_ (1006).jpg',
 'image_ (1007).jpg',
 'image_ (1008).jpg',
 'image_ (1009).jpg',
 'image_ (101).jpg',
 'image_ (1010).jpg',
 'image_ (1011).jpg',
 'image_ (1012).jpg',
 'image_ (1013).jpg',
 'image_ (1014).jpg',
 'image_ (1015).jpg',
 'image_ (1016).jpg',
 'image_ (1017).jpg',
 'image_ (1018).jpg',
 'image_ (1019).jpg',
 'image_ (102).jpg',
 'image_ (1020).jpg',
 'image_ (1021).jpg',
 'image_ (1022).jpg',
 'image_ (1023).jpg',
 'image_ (1024).jpg',
 'image_ (1025).jpg',
 'image_ (1026).jpg',
 'image_ (1027).jpg',
 'image_ (1028).jpg',
 'image_ (1029).jpg',
 'image_ (103).jpg',
 'image_ (1030).jpg',
 'image_ (1031).jpg',
 'image_ (1032).jpg',
 'image_ (1033).jpg',
 'image_ (1034).jpg',
 'image_ (1035).jpg',
 'image_ (1036).jpg',
 'image_ (1037).jpg',
 'image_ (1038).jpg',


In [57]:
import pandas as pd
import time

In [6]:
#from gradio_client import Client
 
#client = Client("http://127.0.0.1:7860/")
#result = client.predict(
#		"https://www.somewebsite.xxx/xxx.jpg",
#		"解释一下这个图片!",	
#		5,	# int | float  in 'Seed' Number component
#		0,	# int | float (numeric value between 0 and 1) in 'top_p' Slider component
#		0,	# int | float (numeric value between 0 and 1) in 'temperature' Slider component
#		fn_index=2
#)
#print(result)

# Check the image quality

In [24]:
import os
from PIL import Image

def check_image_validity(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()  # 验证文件完整性
        return True
    except (IOError, SyntaxError, Image.UnidentifiedImageError):
        return False

valid_images = []
for img_file in os.listdir(image_directory):
    img_path = os.path.join(image_directory, img_file)
    if check_image_validity(img_path):
        valid_images.append(img_path)
    else:
        print(f"损坏文件: {img_path}")

损坏文件: C:/Users/14122/Desktop/test data/Met-Meme/Eimages\image_ (3151).jpg


# Isolate the damaged images

In [29]:
# 在原有代码基础上增加隔离功能
corrupted_dir = os.path.join(image_directory, "corrupted")
os.makedirs(corrupted_dir, exist_ok=True)

for img_file in os.listdir(image_directory):
    img_path = os.path.join(image_directory, img_file)
    if not os.path.isfile(img_path):  # 跳过子目录
        continue
        
    if check_image_validity(img_path):
        valid_images.append(img_path)
    else:
        dest_path = os.path.join(corrupted_dir, img_file)
        os.rename(img_path, dest_path)  # 移动而不是删除
        print(f"已隔离损坏文件: {img_path} -> {dest_path}")

已隔离损坏文件: C:/Users/14122/Desktop/test data/Met-Meme/Eimages\image_ (3151).jpg -> C:/Users/14122/Desktop/test data/Met-Meme/Eimages\corrupted\image_ (3151).jpg


# Try to fix the image

In [34]:
from PIL import Image, ImageFile
import os
import glob

ImageFile.LOAD_TRUNCATED_IMAGES = True  # 强制读取残缺文件

# 定义图像文件夹路径
image_folder = "C:/Users/14122/Desktop/test data/Met-Meme/Eimages/corrupted"
fixed_folder = os.path.join(image_folder, "fixed")

# 创建用于存储修复后图像文件的文件夹
os.makedirs(fixed_folder, exist_ok=True)

# 获取文件夹中所有图像文件的路径
image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))

# 遍历所有图像文件
for image_path in image_paths:
    try:
        # 获取文件名和扩展名
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        
        # 生成新的文件名
        save_path = os.path.join(fixed_folder, f"{name}_fixed{ext}")
        
        with Image.open(image_path) as img:
            img.load()  # 强制加载
            img.save(save_path)  # 另存为新文件
        print(f"修复成功: {base_name}")
    except Exception as e:
        print(f"修复失败: {base_name}, 错误: {e}")

修复成功: image_ (3151).jpg


# Load the Janus-pro-1b api

In [58]:
client = Client("http://127.0.0.1:7860/")
api_info = client.view_api()
print(api_info)

Loaded as API: http://127.0.0.1:7860/ ✔
Client.predict() Usage Info
---------------------------
Named API endpoints: 0

Unnamed API endpoints: 2

 - predict(parameter_3, question, seed, top_p, temperature, fn_index=2) -> response
    Parameters:
     - [Image] parameter_3: str (filepath on your computer (or URL) of image) 
     - [Textbox] question: str 
     - [Number] seed: int | float 
     - [Slider] top_p: int | float (numeric value between 0 and 1) 
     - [Slider] temperature: int | float (numeric value between 0 and 1) 
    Returns:
     - [Textbox] response: str 

 - predict(prompt_prompt_in_more_detail_can_help_produce_better_images, seed_optional, cfg_weight, temperature, fn_index=3) -> generated_images
    Parameters:
     - [Textbox] prompt_prompt_in_more_detail_can_help_produce_better_images: str 
     - [Number] seed_optional: int | float 
     - [Slider] cfg_weight: int | float (numeric value between 1 and 10) 
     - [Slider] temperature: int | float (numeric value bet

# Main Image understanding function

In [40]:
client = Client("http://127.0.0.1:7860/")
results = {}
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    
    # 读取图像文件的二进制数据
    #with open(image_path, "rb") as f:
        #image_data = f.read()
    
    # 调用 client.predict 方法
    result = client.predict(
        image_path,    #filepath
        "Explain this meme, Please use the following format the express the results: <IMAGE_ANALYSIS>; Visual Elements :: 'your description output'; Symbolic Significance :: 'your description output'; Overall Tone :: 'your description output'; <IMAGE_ANALYSIS>",  #question
        42,    #seed
        0.95,   #top_p
        0.1,    #temperature
        fn_index=2    #api_name
    )
    results[image_file] = result
df = pd.DataFrame(list(results.items()), columns=['image_file', 'result'])

output_path = 'C:/Users/14122/Desktop/test data/Met-Meme/rough_data——11111.csv'

df.to_csv(output_path, index=False, encoding='utf-8')

print(f"DataFrame 已成功保存为 {output_path} 文件")

Loaded as API: http://127.0.0.1:7860/ ✔


TypeError: argument of type 'NoneType' is not iterable

# New version with Error handling, Log recording, Retry mechanism, Cache intermediate results to CSV

In [ ]:
#!pip install tenacity

In [66]:
import os
import pandas as pd
from PIL import Image
import time
import logging
from tenacity import retry, stop_after_attempt, wait_fixed
from gradio_client import Client

# 初始化日志配置
logging.basicConfig(
    filename='processing_errors.log',
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# 初始化客户端
client = Client("http://127.0.0.1:7860/")

# 结果保存路径
OUTPUT_CSV = 'C:/Users/14122/Desktop/test data/Met-Meme/test111.csv'

# 定义重试装饰器（3次重试，每次间隔1秒）
@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def safe_predict(image_path):
    return client.predict(
        image_path,
        "Explain this meme, Please use the following format the express the results: <IMAGE_ANALYSIS>; Visual Elements :: 'your description output'; Symbolic Significance :: 'your description output'; Overall Tone :: 'your description output'; <IMAGE_ANALYSIS>",  #question
        42,    #seed
        0.95,   #top_p
        0.1,    #temperature
        fn_index=2    #api_name
    )

def is_image_valid(image_path):
    """验证图片是否损坏"""
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
        logging.error(f"损坏文件: {image_path} - {str(e)}")
        return False

def load_processed_files():
    """加载已处理成功的文件列表"""
    if os.path.exists(OUTPUT_CSV):
        df = pd.read_csv(OUTPUT_CSV)
        return set(df['image_file'].tolist())
    return set()

def save_progress(image_file, result):
    """增量保存结果到CSV"""
    df = pd.DataFrame([{'image_file': image_file, 'result': result}])
    df.to_csv(OUTPUT_CSV, mode='a', header=not os.path.exists(OUTPUT_CSV), index=False, encoding='utf-8')

# 主处理逻辑
def process_images(image_directory):
    processed_files = load_processed_files()
    
    for image_file in os.listdir(image_directory):
        if image_file in processed_files:
            continue
            
        image_path = os.path.join(image_directory, image_file)
        
        # 1. 验证图片完整性
        if not is_image_valid(image_path):
            continue
        
        # 2. 处理并重试
        try:
            result = safe_predict(image_path)
            # 3. 验证响应格式
            if not isinstance(result, str) or "<IMAGE_ANALYSIS>" not in result:
                raise ValueError("无效的响应格式")
                
            # 4. 保存结果
            save_progress(image_file, result)
            print(f"处理成功: {image_file}")
            
        except Exception as e:
            logging.error(f"处理失败: {image_file} - {str(e)}", exc_info=True)
            print(f"处理失败: {image_file}，详见日志")

if __name__ == "__main__":
    image_directory = image_directory  # 替换为实际路径
    process_images(image_directory)
    print(f"处理完成，结果已保存至 {OUTPUT_CSV}")

Loaded as API: http://127.0.0.1:7860/ ✔
处理成功: image_ (3151).jpg
处理完成，结果已保存至 C:/Users/14122/Desktop/test data/Met-Meme/test111.csv


### Check log record

In [60]:
# 读取并显示日志文件内容
log_file_path = 'processing_errors.log'

with open(log_file_path, 'r', encoding='utf-8') as file:
    log_content = file.read()

print(log_content)

2025-03-16 17:20:01,285 - ERROR - 损坏文件: C:/Users/14122/Desktop/test data/Met-Meme/Eimages\corrupted - [Errno 13] Permission denied: 'C:\\Users\\14122\\Desktop\\test data\\Met-Meme\\Eimages\\corrupted'



In [53]:
# 清空日志文件内容 (Optional)
with open(log_file_path, 'w', encoding='utf-8') as file:
    pass

print(f"{log_file_path} 已清空")

processing_errors.log 已清空


In [9]:
import pandas as pd
df = pd.read_csv('C:/Users/14122/Desktop/test data/Met-Meme/test_data.csv')
df.head()

,image_file,result
0,image_ (2614).jpg,<IMAGE_ANALYSIS>\nThe image features a baby wi...
1,image_ (2719).jpg,<IMAGE_ANALYSIS>\nThe image depicts a cartoon ...
2,image_ (2832).jpg,<IMAGE_ANALYSIS>\nThe image shows a black and ...


In [10]:
from IPython.display import display, HTML

pd.set_option('display.max_colwidth', None)

df_head = df.head()

display(HTML(df_head.to_html(index=False)))

image_file,result
image_ (2614).jpg,"<IMAGE_ANALYSIS>\nThe image features a baby with a determined or triumphant expression, wearing a green and white shirt. The background consists of a gradient of blue and purple shades. Overlaid on the image are white text that reads:\n\n""GO TO MCDONALD'S, GET HAPPY MEAL, EXTRA CHICKEN NUGGET.""\n\nThe text is presented in a bold, playful font, which is commonly associated with memes. The baby's expression and the text together create a humorous and lighthearted tone, suggesting a playful command to order food at a fast-food restaurant.\n\nSymbolic Significance:\nThe baby represents a child, and the text humorously mimics a child's command, making the meme relatable and amusing for a younger audience. The phrase ""extra chicken nugget"" is a playful exaggeration of the typical order for a fast-food meal, adding to the humor.\n\nOverall Tone:\nThe tone of the meme is playful and humorous, designed to entertain and evoke a sense of childlike joy."
image_ (2719).jpg,"<IMAGE_ANALYSIS>\nThe image depicts a cartoon cat holding a large stack of plates, with an exaggerated expression of frustration or anger. The text above reads: ""WHEN YO MAMA FORCE YOU TO WASH DISHES OR SHE WILL SMACK."" The visual elements suggest a humorous and exaggerated scenario where the cat is metaphorically ""smashing"" the dishes due to being forced to wash them. The tone is lighthearted and comedic, relying on the absurdity of the situation to elicit laughter.\n</IMAGE_ANALYSIS>"
image_ (2832).jpg,"<IMAGE_ANALYSIS>\nThe image shows a black and white cat with wide, surprised eyes, looking directly at the camera. Above the image, there is a caption: ""When a cat closes its eyes around you it means it trusts you."" Below the image, the text reads: ""My cat:"" followed by the image of the cat.\n\nSymbolic Significance: The image humorously suggests that when a cat closes its eyes around someone, it implies trust and a sense of safety, as cats often do this to protect themselves from potential threats. The exaggerated expression of the cat adds a playful and lighthearted tone to the message.\n\nOverall Tone: The tone is playful and humorous, combining a relatable scenario with a cute visual to create a funny and engaging message.\n</IMAGE_ANALYSIS>"


In [44]:
df_test = df.copy()

In [49]:
def fix_result(result):
    if result.endswith("\n\n<IMAGE_ANALYSIS>"):
        return result.replace("\n\n<IMAGE_ANALYSIS>", "\n</IMAGE_ANALYSIS>")
    if result.endswith("\n<IMAGE_ANALYSIS>"):
        return result.replace("\n<IMAGE_ANALYSIS>", "\n</IMAGE_ANALYSIS>")
    elif result.endswith("."):
        return result + "\n</IMAGE_ANALYSIS>"
    return result

In [50]:
df_test['result'] = df_test['result'].apply(fix_result)

In [51]:
df_test.head(10)

,image_file,result
0,00DjNzR.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a man in white lingerie sitting on a couch, and a woman in a white blouse kneeling beside him. The text overlay reads, ""NOW YOU'RE READY TO VOTE FOR HILLARY."" \nSymbolic Significance: The image appears to be a satirical or humorous take on political commentary, using the visual of a man in lingerie to symbolize a candidate or a specific political figure. The woman's gesture and the text suggest a mock endorsement or a critique of the individual depicted. \nOverall Tone: The tone of the image is humorous and possibly critical, aiming to provoke thought or laughter through its absurdity. \n</IMAGE_ANALYSIS>"
1,05nBEvh.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a person in a formal suit with a confident expression, gesturing with thumbs-up. The background is plain and dark, emphasizing the subject. \nSymbolic Significance: The person appears to be addressing a class, as indicated by the text describing an ethics class where arguments were made about Trump. The gesture of thumbs-up suggests a positive or assertive response to the arguments. \nOverall Tone: The tone is assertive and confident, with a hint of humor or sarcasm in the description of the arguments and the person's reaction.\n</IMAGE_ANALYSIS>"
2,0bOKK62.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image depicts a young woman sitting on a bed in a small, enclosed space, possibly a closet. She is dressed in a formal outfit, and the setting appears to be a bedroom with paneled walls. The woman's posture suggests a sense of contemplation or introspection. \n\nSymbolic Significance: The image evokes themes of vulnerability, secrecy, and self-reflection. The confined space and her attire may symbolize a struggle for independence or a desire to maintain privacy. The setting could represent a personal moment of solitude or a moment of decision-making. \n\nOverall Tone: The tone of the image is somber and introspective, with a hint of melancholy. It conveys a sense of quiet strength and resilience. \n</IMAGE_ANALYSIS>"
3,0EwB4LT.png,"<IMAGE_ANALYSIS> \nVisual Elements: \nThe image is a collage of four panels, each featuring a distinct style of cartoonish or exaggerated character art. The characters are drawn in a simplistic, exaggerated manner, with varying levels of detail and intensity. The first panel shows a calm character with the text ""me me in school."" The second panel features a character with wide eyes and the text ""Bernie Sanders is dumb."" The third panel shows a highly intense character with a screaming expression and the text ""TRUMP!"" The fourth panel is chaotic, with multiple characters and text, including the word ""FUCK"" repeatedly. \n\nSymbolic Significance: \nThe panels collectively convey a sense of frustration, anger, and disbelief, likely reflecting the creator's reaction to political events or statements. The use of exaggerated expressions and bold text emphasizes the emotional intensity of the message. \n\nOverall Tone: \nThe tone is highly negative and critical, with a focus on political commentary and frustration. \n</IMAGE_ANALYSIS>"
4,0JM56ut.png,"<IMAGE_ANALYSIS> \nVisual Elements: \nThe image is a screenshot of a Facebook post. It includes a headline that reads, ""Former Blink-182 Frontman Tom Delonge of blink-182 contacted the government about his UFO research."" Below the headline is a photo of a man speaking into a microphone, presumably Tom Delonge. The post is from a page named ""Diffuser FM."" \n\nSymbolic Significance: \nThe post highlights a notable event involving Tom Delonge, a well-known musician and former frontman of the band Blink-182. The mention of UFO research suggests a connection to extraterrestrial or paranormal topics, which may have sparked public interest or controversy. \n\nOverall Tone: \nThe tone of the post is informative and slightly sensational, as it combines a headline about a significant event with a photo to ca

In [52]:
import pandas as pd
import re

def parse_image_analysis(df):
    pattern = r"""
    <IMAGE_ANALYSIS>\s*
    Visual\s*Elements:\s*(.*?)\s*
    Symbolic\s*Significance:\s*(.*?)\s*
    Overall\s*Tone:\s*(.*?)\s*
    </IMAGE_ANALYSIS>
    """
    
    return df['result'].str.extract(
        re.compile(pattern, flags=re.DOTALL|re.IGNORECASE|re.VERBOSE)
    ).rename(columns={
        0: 'Visual Elements',
        1: 'Symbolic Significance', 
        2: 'Overall Tone'
    }).apply(lambda x: x.str.strip())

In [53]:
new_df = pd.concat([df_test, parse_image_analysis(df_test)], axis=1)

In [55]:
new_df.head(5)

,image_file,result,Visual Elements,Symbolic Significance,Overall Tone
0,00DjNzR.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a man in white lingerie sitting on a couch, and a woman in a white blouse kneeling beside him. The text overlay reads, ""NOW YOU'RE READY TO VOTE FOR HILLARY."" \nSymbolic Significance: The image appears to be a satirical or humorous take on political commentary, using the visual of a man in lingerie to symbolize a candidate or a specific political figure. The woman's gesture and the text suggest a mock endorsement or a critique of the individual depicted. \nOverall Tone: The tone of the image is humorous and possibly critical, aiming to provoke thought or laughter through its absurdity. \n</IMAGE_ANALYSIS>","The image shows a man in white lingerie sitting on a couch, and a woman in a white blouse kneeling beside him. The text overlay reads, ""NOW YOU'RE READY TO VOTE FOR HILLARY.""","The image appears to be a satirical or humorous take on political commentary, using the visual of a man in lingerie to symbolize a candidate or a specific political figure. The woman's gesture and the text suggest a mock endorsement or a critique of the individual depicted.","The tone of the image is humorous and possibly critical, aiming to provoke thought or laughter through its absurdity."
1,05nBEvh.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a person in a formal suit with a confident expression, gesturing with thumbs-up. The background is plain and dark, emphasizing the subject. \nSymbolic Significance: The person appears to be addressing a class, as indicated by the text describing an ethics class where arguments were made about Trump. The gesture of thumbs-up suggests a positive or assertive response to the arguments. \nOverall Tone: The tone is assertive and confident, with a hint of humor or sarcasm in the description of the arguments and the person's reaction.\n</IMAGE_ANALYSIS>","The image shows a person in a formal suit with a confident expression, gesturing with thumbs-up. The background is plain and dark, emphasizing the subject.","The person appears to be addressing a class, as indicated by the text describing an ethics class where arguments were made about Trump. The gesture of thumbs-up suggests a positive or assertive response to the arguments.","The tone is assertive and confident, with a hint of humor or sarcasm in the description of the arguments and the person's reaction."
2,0bOKK62.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image depicts a young woman sitting on a bed in a small, enclosed space, possibly a closet. She is dressed in a formal outfit, and the setting appears to be a bedroom with paneled walls. The woman's posture suggests a sense of contemplation or introspection. \n\nSymbolic Significance: The image evokes themes of vulnerability, secrecy, and self-reflection. The confined space and her attire may symbolize a struggle for independence or a desire to maintain privacy. The setting could represent a personal moment of solitude or a moment of decision-making. \n\nOverall Tone: The tone of the image is somber and introspective, with a hint of melancholy. It conveys a sense of quiet strength and resilience. \n</IMAGE_ANALYSIS>","The image depicts a young woman sitting on a bed in a small, enclosed space, possibly a closet. She is dressed in a formal outfit, and the setting appears to be a bedroom with paneled walls. The woman's posture suggests a sense of contemplation or introspection.","The image evokes themes of vulnerability, secrecy, and self-reflection. The confined space and her attire may symbolize a struggle for independence or a desire to maintain privacy. The setting could represent a personal moment of solitude or a moment of decision-making.","The tone of the image is somber and introspective, with a hint of melancholy. It conveys a sense of quiet strength and resilience."
3,0EwB4LT.png,"<IMAGE_ANALYSIS> \nVisual Elements: \nThe image is a collage of fou

In [57]:
has_nan = new_df.isnull().values.any()
print(f"If DataFrame has NaN value: {has_nan}")

If DataFrame has NaN value: True


In [58]:
nan_rows = new_df[new_df.isnull().any(axis=1)]
print("Rows with NaN values:")
display(HTML(nan_rows.to_html(index=False)))

Rows with NaN values:


image_file,result,Visual Elements,Symbolic Significance,Overall Tone
bHyYl3j.png,"<IMAGE_ANALYSIS>; Visual Elements: The image is a humorous meme that uses a voting system to compare three individuals: Hillary Clinton, Donald Trump, and Trebuchiet. Each individual has a photo, and below them are statements about their actions or statements. Voting symbols (green checkmarks and red crosses) indicate whether the statement is true or false.\n\nSymbolic Significance: The meme humorously critiques the individuals' statements and actions. Hillary Clinton and Donald Trump have green checkmarks, indicating that their statements are considered true. Trebuchiet, represented by a trebuchet, has a red cross for the first statement and a green checkmark for the second, suggesting a mixed evaluation.\n\nOverall Tone: The tone is lighthearted and satirical, using the voting system to make a playful commentary on the individuals' statements.\n</IMAGE_ANALYSIS>",NaN,NaN,NaN
Cco_BiaUMAAla6m.jpg,"<IMAGE_ANALYSIS> \nVisual Elements: \n- The image is a meme featuring two individuals, one on the left and one on the right, with a humorous caption at the top. \n- The left side shows a person with a serious expression, wearing a gold chain, and the text below reads: ""You can't fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony a fake a phony",NaN,NaN,NaN
GbhHJH6.png,"<IMAGE_ANALYSIS>; Visual Elements: The image depicts a cartoon-like character with a distorted face, wearing a green shirt with a logo that reads ""FEEL THE SEER."" The character appears to be speaking to another figure, who is partially visible on the right side of the image. The background is plain white, and there is a speech bubble coming from the character on the left.\n\nSymbolic Significance: The image seems to convey a message about racial and social issues, particularly focusing on the experiences of individuals who have been attacked. The distorted face and the character's appearance may symbolize the emotional or psychological impact of such events. The text in the speech bubble suggests a narrative about racial profiling and the frustration of being targeted by authorities based on racial biases.\n\nOverall Tone: The tone of the image is critical and confrontational, addressing issues of racial injustice and the emotional toll of such experiences. The use of exaggerated and distorted imagery adds to the intensity and urgency of the message.\n</IMAGE_ANALYSIS>",NaN,NaN,NaN
gy3AJXJ.png,"<IMAGE_ANALYSIS>; Visual Elements: The image features a political debate poster with the title ""Bernie or Hillary?"" and the tagline ""Be informed. Compare them on the issues that matter."" Below the title, there are two portraits of individuals, presumably candidate

In [31]:
def parse_image_analysis(df):
    pattern = r"""
    <IMAGE_ANALYSIS>\s*
    Visual\s*Elements?:\s*(.*?)(?=\s*Symbolic\s*Significance:|\s*</IMAGE_ANALYSIS>)  # 使用预查确保边界[1]
    \s*Symbolic\s*Significance:\s*(.*?)(?=\s*Overall\s*Tone:|\s*</IMAGE_ANALYSIS>)
    \s*Overall\s*Tone:\s*(.*?)\s*
    </IMAGE_ANALYSIS>
    """
    
    return df['result'].str.extract(
        re.compile(pattern, flags=re.DOTALL|re.IGNORECASE|re.VERBOSE)
    ).rename(columns={
        0: 'Visual Elements',
        1: 'Symbolic Significance',
        2: 'Overall Tone'
    }).apply(lambda x: x.str.strip())

In [32]:
df_test_a = df.copy()

In [33]:
new_df_a = pd.concat([df_test_a, parse_image_analysis(df_test_a)], axis=1)

In [35]:
new_df_a.head(50)

,image_file,result,Visual Elements,Symbolic Significance,Overall Tone
0,00DjNzR.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a man in white lingerie sitting on a couch, and a woman in a white blouse kneeling beside him. The text overlay reads, ""NOW YOU'RE READY TO VOTE FOR HILLARY."" \nSymbolic Significance: The image appears to be a satirical or humorous take on political commentary, using the visual of a man in lingerie to symbolize a candidate or a specific political figure. The woman's gesture and the text suggest a mock endorsement or a critique of the individual depicted. \nOverall Tone: The tone of the image is humorous and possibly critical, aiming to provoke thought or laughter through its absurdity. \n<IMAGE_ANALYSIS>",NaN,NaN,NaN
1,05nBEvh.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image shows a person in a formal suit with a confident expression, gesturing with thumbs-up. The background is plain and dark, emphasizing the subject. \nSymbolic Significance: The person appears to be addressing a class, as indicated by the text describing an ethics class where arguments were made about Trump. The gesture of thumbs-up suggests a positive or assertive response to the arguments. \nOverall Tone: The tone is assertive and confident, with a hint of humor or sarcasm in the description of the arguments and the person's reaction.",NaN,NaN,NaN
2,0bOKK62.png,"<IMAGE_ANALYSIS> \nVisual Elements: The image depicts a young woman sitting on a bed in a small, enclosed space, possibly a closet. She is dressed in a formal outfit, and the setting appears to be a bedroom with paneled walls. The woman's posture suggests a sense of contemplation or introspection. \n\nSymbolic Significance: The image evokes themes of vulnerability, secrecy, and self-reflection. The confined space and her attire may symbolize a struggle for independence or a desire to maintain privacy. The setting could represent a personal moment of solitude or a moment of decision-making. \n\nOverall Tone: The tone of the image is somber and introspective, with a hint of melancholy. It conveys a sense of quiet strength and resilience. \n</IMAGE_ANALYSIS>","The image depicts a young woman sitting on a bed in a small, enclosed space, possibly a closet. She is dressed in a formal outfit, and the setting appears to be a bedroom with paneled walls. The woman's posture suggests a sense of contemplation or introspection.","The image evokes themes of vulnerability, secrecy, and self-reflection. The confined space and her attire may symbolize a struggle for independence or a desire to maintain privacy. The setting could represent a personal moment of solitude or a moment of decision-making.","The tone of the image is somber and introspective, with a hint of melancholy. It conveys a sense of quiet strength and resilience."
3,0EwB4LT.png,"<IMAGE_ANALYSIS> \nVisual Elements: \nThe image is a collage of four panels, each featuring a distinct style of cartoonish or exaggerated character art. The characters are drawn in a simplistic, exaggerated manner, with varying levels of detail and intensity. The first panel shows a calm character with the text ""me me in school."" The second panel features a character with wide eyes and the text ""Bernie Sanders is dumb."" The third panel shows a highly intense character with a screaming expression and the text ""TRUMP!"" The fourth panel is chaotic, with multiple characters and text, including the word ""FUCK"" repeatedly. \n\nSymbolic Significance: \nThe panels collectively convey a sense of frustration, anger, and disbelief, likely reflecting the creator's reaction to political events or statements. The use of exaggerated expressions and bold text emphasizes the emotional intensity of the message. \n\nOverall Tone: \nThe tone is highly negative and critical, with a focus on political commentary and frustration. \n\n<IMAGE_ANALYSIS>",NaN,NaN,NaN
4,0JM56ut.png,"<IMAGE_ANALYSIS> \nVisual Elements: \nThe image is a screenshot of a Faceb

In [33]:
start_time = time.time()
client = Client("http://127.0.0.1:7860/")
results = {}
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    
    # 读取图像文件的二进制数据
    #with open(image_path, "rb") as f:
        #image_data = f.read()
    
    # 调用 client.predict 方法
    result = client.predict(
        image_path,    #filepath
        "Explain this meme, Please use the following format the express the results: <IMAGE_ANALYSIS>; Visual Elements :: 'your description output'; Symbolic Significance :: 'your description output'; Overall Tone :: 'your description output'; <IMAGE_ANALYSIS>",  #question
        42,    #seed
        0.95,   #top_p
        0.1,    #temperature
        fn_index=2    #api_name
    )
    results[image_file] = result
    end_time = time.time()
    print(f"Result for {image_file}: {result}")

Loaded as API: http://127.0.0.1:7860/ ✔
Result for ca18ac59e5563d609aef4f0349b406e6.jpg: <IMAGE_ANALYSIS>  
Visual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase "大坏蛋熊猫" (translated as "Big bad panda") and "一个坏人" (translated as "A bad person").  

Symbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers.  

Overall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction.  
</IMAGE_ANALYSIS>
Result for OIP.jpeg: <IMAGE_ANALYSIS>  
Visual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features

In [ ]:
#start_time = time.time()
#
#client = Client("http://127.0.0.1:7860/")
#results = {}
#for image_file in image_files:
#    image_path = os.path.join(image_directory, image_file)
#    result = client.predict(
#        image=handle_file(image_path),
#        question="explain this meme, show the results into two part, one for analysis of the image, the other one for general explanation of the meme",
#        seed=42,
#        top_p=0.95,
#        temperature=0.1,
#        api_name="/multimodal_understanding"
    )
#    results[image_file] = result
#    print(f"Result for {image_file}: {result}")
#
#end_time = time.time()



# Print the runtime

In [34]:
end_time = time.time()
runtime = end_time - start_time
print(f"Total runtime: {runtime:.2f} seconds")

Total runtime: 181.68 seconds


In [35]:
results

{'ca18ac59e5563d609aef4f0349b406e6.jpg': '<IMAGE_ANALYSIS>  \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase "大坏蛋熊猫" (translated as "Big bad panda") and "一个坏人" (translated as "A bad person").  \n\nSymbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers.  \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction.  \n</IMAGE_ANALYSIS>',
 'OIP.jpeg': '<IMAGE_ANALYSIS>  \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred 

In [36]:
import pandas as pd
df = pd.DataFrame(list(results.items()), columns=['image_file', 'result'])

In [37]:
print (df)

                             image_file  \
0  ca18ac59e5563d609aef4f0349b406e6.jpg   
1                              OIP.jpeg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           result  
0                                                                                <IMAGE_ANALYSIS> 

In [38]:
import pandas as pd
from IPython.display import display, HTML

pd.set_option('display.max_colwidth', None)

display(HTML(df.to_html(index=False)))

image_file,result
ca18ac59e5563d609aef4f0349b406e6.jpg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person""). \n\nSymbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers. \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction. \n</IMAGE_ANALYSIS>"
OIP.jpeg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change."" \n\nSymbolic Significance: The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea. \n\nOverall Tone: The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer. \n</IMAGE_ANALYSIS>"


In [44]:
df_test = df.copy()

In [45]:
df_test

,image_file,result
0,ca18ac59e5563d609aef4f0349b406e6.jpg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person""). \n\nSymbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers. \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction. \n</IMAGE_ANALYSIS>"
1,OIP.jpeg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change."" \n\nSymbolic Significance: The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea. \n\nOverall Tone: The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer. \n</IMAGE_ANALYSIS>"


In [41]:

# 定义分割函数
def split_result(result):
    visual_elements_start = result.find("Visual Elements'") + len("Visual Elements")
    visual_elements_end = result.find("Symbolic Significance")
    symbolic_significance_start = visual_elements_end + len("symbolic significance")
    symbolic_significance_end = result.find("Overall tone")
    overall_tone_start = symbolic_significance_end + len("Overall tone")
    overall_tone_end = result.find("' </IMAGE_ANALYSIS>")

    visual_elements = result[visual_elements_start:visual_elements_end]
    symbolic_significance = result[symbolic_significance_start:symbolic_significance_end]
    overall_tone = result[overall_tone_start:overall_tone_end]

    return pd.Series([visual_elements, symbolic_significance, overall_tone])

# 应用分割函数
df_test[['Visual Elements', 'Symbolic Significance', 'Overall Tone']] = df_test['result'].apply(split_result)

# 删除原始列
df_test.drop(columns=['result'], inplace=True)

# 显示结果
print(df_test)

                             image_file  \
0  ca18ac59e5563d609aef4f0349b406e6.jpg   
1                              OIP.jpeg   

                                                                                                                                                                                                                                                                                                                                         Visual Elements  \
0                                              S>  \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase "大坏蛋熊猫" (translated as "Big bad panda") and "一个坏人" (translated as "A bad person").  \n\n   
1  S>  \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, wi

In [42]:
display(HTML(df_test.to_html(index=False)))

image_file,Visual Elements,Symbolic Significance,Overall Tone
ca18ac59e5563d609aef4f0349b406e6.jpg,"S> \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person""). \n\n",": The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers. \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction. \n</IMAGE_ANALYSIS","YSIS> \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person""). \n\nSymbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers. \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction. \n</IMAGE_ANALYSIS"
OIP.jpeg,"S> \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change."" \n\n",": The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea. \n\nOverall Tone: The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer. \n</IMAGE_ANALYSIS","YSIS> \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change."" \n\nSymbolic Significance: The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea. \n\nOverall Tone: The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer. \n</IMAGE_ANALYSIS"


In [43]:
import pandas as pd
import re

def parse_image_analysis(df):
    pattern = r"""
    <IMAGE_ANALYSIS>\s*
    Visual\s*Elements:\s*(.*?)\s*
    Symbolic\s*Significance:\s*(.*?)\s*
    Overall\s*Tone:\s*(.*?)\s*
    </IMAGE_ANALYSIS>
    """
    
    return df['result'].str.extract(
        re.compile(pattern, flags=re.DOTALL|re.IGNORECASE|re.VERBOSE)
    ).rename(columns={
        0: 'Visual Elements',
        1: 'Symbolic Significance', 
        2: 'Overall Tone'
    }).apply(lambda x: x.str.strip())

In [46]:
new_df = pd.concat([df_test, parse_image_analysis(df_test)], axis=1)

In [47]:
display(HTML(new_df.to_html(index=False)))

image_file,result,Visual Elements,Symbolic Significance,Overall Tone
ca18ac59e5563d609aef4f0349b406e6.jpg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person""). \n\nSymbolic Significance: The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers. \n\nOverall Tone: The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction. \n</IMAGE_ANALYSIS>","The image features a stylized panda face with a human-like expression, characterized by a mischievous or sarcastic smile. The text below the image is written in Chinese, with the phrase ""大坏蛋熊猫"" (translated as ""Big bad panda"") and ""一个坏人"" (translated as ""A bad person"").","The panda face is a popular internet meme, often used to convey sarcasm, humor, or irony. The human-like expression adds a layer of absurdity or humor, making the image relatable or amusing to viewers.","The tone of the image is humorous and sarcastic, likely intended to provoke laughter or a lighthearted reaction."
OIP.jpeg,"<IMAGE_ANALYSIS> \nVisual Elements: The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change."" \n\nSymbolic Significance: The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea. \n\nOverall Tone: The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer. \n</IMAGE_ANALYSIS>","The image features a panda with a human face superimposed on its head, holding the face in place with its paws. The human face appears distorted, with exaggerated features such as a wide smile and a slightly blurred appearance. The text below the image reads ""一秒变脸,"" which translates to ""One-second Face Change.""","The image humorously represents the concept of rapidly changing facial expressions, often used in memes to depict sudden shifts in mood or personality. The exaggerated distortion of the face adds a comedic effect, emphasizing the absurdity of the idea.","The tone of the image is lighthearted and humorous, intended to entertain and amuse the viewer."
